<div style='text-align: center;'>
<img src="images/math60082-banner.png" alt="image" width="80%" height="auto">
</div>

# Lab Class - Week 2
## Dr P. V. Johnson
## Department of Mathematics

# Last Week

- Coding Support Classes
- Previous class: [Coding Support Class - Part 2](https://github.com/pjohno/MATH60082-python/blob/main/MATH60082-coding-support-class-2.ipynb)
- Solutions: [Coding Workbook 2 Solutions](https://github.com/pjohno/MATH60082-python/blob/main/solutions/SLTN-coding-workbook-2.ipynb)


# This Week

- How to solve problems with code
- Demo 2.1 - Integrating a normal distribution
- Demo 2.2 - How to evaluate the efficiency of code
- Demo 2.3 - Coursework Example

## Solving Problem with Computers

- First we think about the stages of our program.  
- What are the inputs to the program?
- What tools do we need? What algorithms need coding up? 
- What the required outputs of the program?
- How are we going to use the results?

Imagine we want to import real data (from the web) and calibrate it against the Black-Scholes model.

- Most people make the mistake of trying to write a program to do everything at once
- Instead you should split you program into several different tasks
- Each task should be coded up individually and tested
- Task 1: Read data from the web 
- Task 2: Calibrate Black-Scholes against test data
- Task 3: Combine results of Task 1 and Task 2

# Demo 2.1: Calculating Knock-out Barrier Option

The analytic solution to the value of a Knock-Out Option $G$ with a barrier at $x$ is given by 
$$
G(x)=\frac{1}{\sqrt{2\pi}} \int_{-\infty}^{x} e^{-t^2/2} g(t) dt
$$
where $g(t)$ is the payoff function for the option.

In the case $g(t)\equiv 1$, $G(x)$ becomes the Cumulative Normal Distribution function $G(x)=N(x)$ where
$$
N(x)=\frac{1}{\sqrt{2\pi}} \int_{-\infty}^{x} e^{-t^2/2}dt .
$$

Find the value of a Knock-Out Barrier Option $G(0.75)$ when 
$$
g(t)= \frac{1}{1+t^2}
$$
using numerical integration.

- First we think about the stages of our program.
- We must choose a method to implement the integration, test and verify it. 
- Once it is then applied to the problem we need to decide how to deal with the concept of infinity in this setting. 

# Benchmarking

- To create fast and efficient python code the primary focus should be on adapting existing low level routines and algorithms to your problem
- We normally test coding methods on problems with known solutions so we can see how errors behave, and how efficient the code is
- In this case $g\equiv 1$, the module `scipy` has a collection of special functions, of which $N(x)$ above is one.
- The function is inside the `scipy.special` module called `ndtr`. We can import it and define a new name for it `ND` as follows:

In [1]:
from scipy.special import ndtr as ND 

benchMarkRresult = ND(1.)
benchMarkRresult

0.8413447460685429

- The above code can be used as our benchmark, it is likely that this will be the most efficient way to calculate $N(x)$ but we will check for definite later.

# Coding the Numerical Integration

- Low level algorithms implementing numerical integrations should use existing routines
- Our job is to stage our path to a final solution and check everything works along the way
- First task is to look up integration methods and solve the simple problem we already have a solution for when $g\equiv1$
- Please see the documentation for more details, [https://docs.scipy.org/doc/scipy/tutorial/integrate.html](https://docs.scipy.org/doc/scipy/tutorial/integrate.html), the method we want to use here is `quad` from the module `scipy.integrate`.

## Stage 1

- First lets import the libraries and solve problem with finite limits
$$
I = \frac{1}{\sqrt{2\pi}}\int_0^1 e^{-\frac{t^2}{2}} dt
$$
- we need to create the integrand function to pass as an argument -- how to do this?
- Typically we use _lambda_ functions, the syntax is
~~~
< lambda function > = lambda <variables>: <function definition>
~~~
- so to create the integrand we could write
~~~
integrand = lambda t: exp(-t*t/2.)
~~~
- you can then call it as a function
~~~
integrand(0.5)
~~~

In [2]:
from math import exp
integrand = lambda t: exp(-t*t/2.)
integrand(0.5)

0.8824969025845955

- first let's just integrate the integrand, calling the quad function on limits 0 to 1

In [3]:
from scipy.integrate import quad as QUAD
from math import exp

integrand = lambda t: exp(-t*t/2.)

I = QUAD(integrand , 0, 1)
I

(0.855624391892149, 9.499339003095619e-15)

Notice that `I` returns two numbers, the actual value we want and the error. Pick out the value by writing `[0]` at the end, so

In [4]:
from math import exp,pi,sqrt
I = (1./sqrt(2.*pi))*QUAD(integrand , 0, 1)[0]
I

0.34134474606854304

# Tasks

- Integrate $f(x)=\sin(x)$ and $f(x)=\cos(x)$ over the region $\left[0,\frac{3}{4}\pi\right]$. Verify the accuracy of your results.
- Consider that you are required to integrate the function $f(x)=\max(x,e^\frac{x}{2}-1)$ over the region $[0,5]$. How might you best deal with this problem?

## Stage 2

Next we need to deal with negative infinity on the lower limit. For simple case $N(x)$ we can make use of the fact $N(0)=\frac12$ and the properties of integrals to say
$$
N(x) = \frac12 + \frac{1}{\sqrt{2\pi}}\int_0^1 e^{-\frac{t^2}{2}} dt
$$
This gives

In [5]:
x = 1
Nx = 0.5 + (1./sqrt(2.*pi))*QUAD(integrand , 0, x)[0]
Nx

0.841344746068543

Checking against the benchmark we get a tiny difference between the numbers

In [6]:
Nx - benchMarkRresult

1.1102230246251565e-16

This works well for $-15\leq x \leq 15$, but for $x$ outside this range we know $N\approx 0$ for $x<-15$ and $N\approx 1$ for $x>15$. If you are writing a function, you should deal with cases before wasting time doing any calculations. 

In [7]:
# function to integrate cumulative normal distribution
def Nx_integrate( x ):
    if x<-15.0:
        return 0.
    elif x>15.0:
        return 1.0
    from scipy.integrate import quad as QUAD
    from math import exp,pi,sqrt
    return 0.5 + (1./sqrt(2.*pi))*QUAD(lambda t: exp(-t*t/2.), 0, x)[0]

Nx_integrate(1.)

0.841344746068543

# Stage 3

Now let's adapt the code to take account of $g(t)=(1+t^2)^{-1}$. Copy the function definition above and change the name to `Gx_integrate`. For the lambda function, include the extra term required, it should now look like:

In [8]:
# function to integrate normal distribution multiplied by 1/(1+t^2)
def Gx_integrate( x ):
    lower_limit=-15
    upper_limit=15
    if x<lower_limit:
        return 0.
    elif x>upper_limit:
        return Gx_integrate(upper_limit)
    from scipy.integrate import quad as QUAD
    from math import exp,pi,sqrt
    return (1./sqrt(2.*pi))*QUAD(lambda t: exp(-t*t/2.)/(1+t*t) , lower_limit, x)[0]

Gx_integrate(1.)

0.6036660402687406

This function assumes we know for definite what form the function $g$ takes. In practice this might not be the case. In the next section we will investigate the efficiency of the code.

# Tasks:

- Experiment with the lower and upper limits to see what effect they have. Can you propose what would be the _best_ values to choose in this case? Explain your reasoning.
- Try to write a function `Gxg_integrate( float: x , g )` which takes the `g` as an argument.

# Demo 2.2: Timing your codes

## Efficiency

Efficiency refers to the ability to accomplish a task with the least amount of resources or effort, maximising output while minimising input. In the context of this course unit, we wish to:
- **maximise** the accuracy, and
- **minimise** the computation time.

Accuracy is a measure of the difference between the true solution and our approximation. The smaller the difference the better, so if
$$
\text{Method A} - \text{True Solution} = 0.001
$$
and
$$
\text{Method B} - \text{True Solution} = 0.00001
$$
we can say Method B is more accurate. However, if Method B takes significantly longer to calculate Method A, it might not be _more_ efficient.

If the difference in accuracy between the different functions above is close to zero, to test the efficiency of the code we only need to look at the computation times. The fastest code in this case will also be the most efficient.

# Tasks:

- Test the efficiency of calculation for $N(x)$ against the version from the special function module in `scipy`.
- Test the efficiency of calculating $G(x)$ when $g$ is written inside the function, versus when $g$ is passed in as an argument. Does the flexibility of the second method come at a cost?

## Timeit Module

To test the efficiency of our code, clearly we need to check the time it takes to run. To do this we suggest using the function `timeit`. To import the function, write:
~~~
from timeit import timeit
~~~
and to call it the syntax is as follows:
~~~
<time taken to run> = timeit( <a string containing the python script to run>, number=<iterations>, globals=globals())
~~~
So for our example we can write



In [9]:
from timeit import timeit

In [10]:
n = 100000
script="ND(1.)"
timeIntegrate = timeit( script,number=n,globals=globals() )

print("Time taken to run ",n," calls to the function ",script, " is ", timeIntegrate," seconds.")

Time taken to run  100000  calls to the function  ND(1.)  is  0.08505876299750526  seconds.


In [11]:

n = 100000
script="Nx_integrate(1.)"
timeIntegrate = timeit( script,number=n,globals=globals() )

print("Time taken to run ",n," calls to the function ",script, " is ", timeIntegrate," seconds.")

Time taken to run  100000  calls to the function  Nx_integrate(1.)  is  1.0715042580013687  seconds.


Now try to do this on some of the other functions.

# Demo 2.3: Black-Scholes Solution

A trader has asked you to price the value of the call option $C(S,t)$ at time $t = 0$ according to the standard Black-Scholes formula, where $T=1$, $X=1$, $r=0.05$ and $\sigma=0.2$. Write a program to calculate 
$C$ and output the results to screen. You must generate four columns of data:
- the value of $S$;
- the value of $d_1$;
- the value of $d_1$;
- the value of $C(S,t=0)$.

Output each of the values when the stock price is
$$
S\in\{ 0.8,0.9,1,1.1,1.2\} .
$$
You should use a `for` loop to generate the data.


# Tasks:

- write a function that returns the value of a call option using standard Black-Scholes formula
- write a loop to return the values at different values of $S$
- format the table into latex

First we need the cumulative standard normal distribution, the version in `scipy.special` was identified as the fastest way to calculate this.

In [12]:
from scipy.special import ndtr as ND 

Declare our values for $T$, $X$, $r$ and $\sigma$, and then try to calculate the values of $d_1$ and $d_2$ from the formula, using an arbitrary value of $S=0.9$.

In [13]:
from math import log,exp,sqrt
T=1.0
X=1.0
r=0.05
sigma=0.2
S=0.9

d1 = ( log(S/X) + (r+sigma*sigma/2.0)*T)/sigma/sqrt(T)
d2 = ( log(S/X) + (r-sigma*sigma/2.0)*T)/sigma/sqrt(T)

print("Values d1=",d1," and d2=",d2)

Values d1= -0.17680257828913137  and d2= -0.3768025782891314


We can also calculate the value of $N(d_1)$ and $N(d_2)$:

In [14]:
print("Values N(d1)=",ND(d1)," and ND(d2)=",ND(d2))

Values N(d1)= 0.42983173188954316  and ND(d2)= 0.35316016226972924


So our call option function may look something like

In [15]:
def callOptionBS(S,X,T,r,sigma):
    d1 = ( log(S/X) + (r+sigma*sigma/2.0)*T)/sigma/sqrt(T)
    d2 = ( log(S/X) + (r-sigma*sigma/2.0)*T)/sigma/sqrt(T)
    return S*ND(d1) - X*exp(-r*T)*ND(d2)

and putting this in a loop we get

In [16]:
for i in range(8,13):
    S = i*0.1
    d1 = ( log(S/X) + (r+sigma*sigma/2.0)*T)/sigma/sqrt(T)
    d2 = ( log(S/X) + (r-sigma*sigma/2.0)*T)/sigma/sqrt(T)
    C=callOptionBS(S,X,T,r,sigma)
    print(S," ",d1," ",d2," ",C)

0.8   -0.7657177565710485   -0.9657177565710485   0.018594195728121904
0.9   -0.17680257828913137   -0.3768025782891314   0.05091222078817553
1.0   0.35000000000000003   0.15   0.10450583572185568
1.1   0.8265508990216247   0.6265508990216246   0.17662953740590448
1.2000000000000002   1.261607783969774   1.061607783969774   0.2616904394684735


When formatted into latex, the table should look something like this:
<div style='text-align: left;'>
<img src="images/lab-demo-2-3.png" alt="image" width="60%" height="auto">
</div>

Please see this [Overleaf Project](https://www.overleaf.com/read/psctxvmfbcfw#dd111b) for an example of how to use this output in a LaTeX document.

You can use f-strings to shorten the numbers. The syntax is
~~~
f" {variable_name:<formatting flags>} "
~~~
Inside the braces, put the variable name or expression, and the formatting flags after `:` set width and precision. For example: `f" f(x):={fx:12.5f} "`. This uses the same formatting mini-language as `.format(...)`, but keeps the value inline. Some examples below:-

In [17]:
# create 12 blank spaces and format a real number using 5 decimal places
v = 124.150984361249124
str = f"|v:={v:12.5f}|"
print(str)
# create 5 blank spaces and format an integer
i = int(124)
str = f"|i:={i:5d}|"
print(str)
# multiple variables are entered in the order they appear
str = f"|i:={i:5d}|v:={v:12.5f}|"
print(str)

|v:=   124.15098|
|i:=  124|
|i:=  124|v:=   124.15098|


See [https://pyformat.info/](https://pyformat.info/) for more details.

## Next steps

- Download the workbook: [MATH60082-lab-workbook-2.ipynb](https://github.com/pjohno/MATH60082-python/blob/main/MATH60082-lab-workbook-2.ipynb)
- Next class notebook: [MATH60082-lab-class-3.ipynb](https://github.com/pjohno/MATH60082-python/blob/main/MATH60082-lab-class-3.ipynb)
